In [ ]:
from pathlib import Path
import pandas as pd

In [ ]:
f = Path("Data/alpaca_data.csv")
df = pd.read_csv(f, parse_dates = True, infer_datetime_format = True)
df.head()


In [ ]:
import ml
import nlp

In [ ]:
df = nlp.fix_date(df)
l = ml.LSTM(df)
dfp = l.predict(1,1)

dfp.head()


In [ ]:
from nlp import NLT
n = NLT()
new_df = n.make_sentiment_df(df,"twits")
new_df.head()

In [ ]:
b = nlp.Blobby()

new_df.head()
new_new_df = b.add_blob(new_df, "Text")
new_new_df.head()

In [ ]:
# l = ml.LSTM()

# l.num_units
# plot_df = new_new_df[["Text_Compound","Text_Neg", "Text_Pos"]]
# l.plot_me(plot_df)

In [ ]:
sent_df = nlp.make_sentiment_df(df, "Reddit")

In [ ]:
sent_df.head()

In [ ]:
nlp.show_stats(sent_df)